In [1]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from os.path import exists
import random

#GP
import sklearn
from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import PredictionErrorDisplay


#LHC sampling 
from scipy.stats import qmc

#map
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

#homemade function file
import importlib
import ppe_loading_functions #as functions
importlib.reload(ppe_loading_functions)
import ref_data_functions #as functions
importlib.reload(ref_data_functions)


import matplotlib.colors as mcolors
mcolors.CSS4_COLORS

colors=['c', 'b', 'm', 'k', 'y', 'gray','lightcoral', 'dodgerblue', 'blueviolet']
shapes=['.', '*', '+', '>', '<', 's','-.', 'o']

# Load ICON PPEs

In [2]:
var1stPPE, results1stPPE,var2ndPPE, results2ndPPE, varMoreCloudParam, resultsMoreCloudParam,\
 varMoreCloudParam2, resultsMoreCloudParam2, varPhyDyn_NewParam_2tun,resultsPhyDyn_NewParam_2tun,\
 varPhyDyn_NewParam_5thPPE_2tun,results_5thPPE_2tun=\
ppe_loading_functions.load_all_ppes()

In [3]:
#Ref tuned ICON-A 2.6.4
pressure_addi,pressure_fact,tauufact,aes_glo_psl,aes_SOO_tauu,aes_NAO_tauu,aes_60N_1000hPa_ua,aes_60S_1000hPa_ua,\
aes_60N_10hPa_ua,aes_60S_10hPa_ua=ref_data_functions.ICON_aes_refdata()

In [4]:
#Physics Ref
ref_mean, ref_std=ref_data_functions.References_outputs()
#Dynamics Ref
refsERA5, refsMERRA2, refsERAInterim=ref_data_functions.References_Dynoutputs()

#all ref values for Physics and Dynamic variables
ref2tun_mean=np.zeros(9)
ref2tun_mean[0:5]=ref_mean #physics variables
ref2tun_mean[5:]=[refsERA5[0],refsERA5[1],refsERA5[2],refsERA5[3]] #dynamics variables
#ref_mean
print(ref2tun_mean)

#all std for Physics and Dynamic variables: not used yet in the HM flow

/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:26: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  ref_TOASW=file['gsolar_clim'][:]-file['gtoa_sw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:26: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  ref_TOASW=file['gsolar_clim'][:]-file['gtoa_sw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:27: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  ref_TOALW=-file['gtoa_lw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:27: UserWarnin

[ 2.40866470e+02 -2.40114889e+02  5.00000000e-01  6.74343643e+01
  2.44086728e+01  1.09413767e+01  3.27732849e+01  2.94719753e-03
  1.36748120e-01]


/home/b/b309216/kernels/my-kernel/lib/python3.8/site-packages/iris/analysis/cartography.py:412: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [5]:
ref_Clt_ESACCI, ref_Clt_ESACCI_std, ref_Clt_CLARA_AVHRR, ref_Clt_CLARA_AVHRR_std=ref_data_functions.Ref_clt()
ref_Prw_ESACCI, ref_Prw_ERA5, ref_Prw_ERA5_std=ref_data_functions.Ref_prw()

In [6]:
years_clt_esacci,clt_esacci_per_year=ref_data_functions.clt_esacci_per_years()
years_clt_claraavhrr,clt_claraavhrr_per_year=ref_data_functions.clt_claraavhrr_per_years()
years_prw_era5,prw_era5_per_year=ref_data_functions.prw_era5_per_years()

In [7]:
#reference data 2nd tuning step
ref2tun_mean=ref_data_functions.era5_data()

/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:26: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  ref_TOASW=file['gsolar_clim'][:]-file['gtoa_sw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:26: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  ref_TOASW=file['gsolar_clim'][:]-file['gtoa_sw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:27: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  ref_TOALW=-file['gtoa_lw_all_clim'][:] #m/W^2
/home/b/b309216/bonnet23_gmd_automatic_tuning_atm_paper/Paper_clean_code/D-read_and_analyze_output_data/ref_data_functions.py:27: UserWarnin

# History Matching on the 1st tuning step output metrics

In [19]:
#bounds enlarged for LHC (same as in other jupyter notebook /!\)
Emid_bounds = [2e-5, 3e-4]
Epen_bounds = [2e-5, 6e-4]
Edd_bounds = [5e-5, 6e-4]
Taumf_bounds = [200, 10000]
Pr_bounds = [5e-1, 1.2]
R0top_bounds = [5e-1, 9e-1]
bounds=[Emid_bounds, Epen_bounds, Edd_bounds, Taumf_bounds, Pr_bounds, R0top_bounds]

In [17]:
# check the uncertainty of the chosen sample compared to the distribution of uncertainty, + absolute error 

# train the GP on the training sample
kernel = 1 * Matern(length_scale=1.0, nu=1.5) #RBF(length_scale=1.0, length_scale_bounds=(1e-5, 1e5)) #np.array([1.0,1.0,1.0])) #length_scale=10) # ,1.0,1.0,1.0]))#, length_scale_bounds=(1e-2, 1e2))
gaussian_process_multi = GaussianProcessRegressor(kernel=kernel,alpha=1e-5, n_restarts_optimizer=20)

#scale data https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
scaler_var1 = preprocessing.StandardScaler().fit(var1stPPE)
scaled_var1 = scaler_var1.transform(var1stPPE)

gaussian_process_multi.fit(scaled_var1, results1stPPE) #
gaussian_process_multi.kernel_

102**2 * Matern(length_scale=70.4, nu=1.5)

In [20]:
# run the GP on a LHC sample of BIG SIZE
from scipy.stats import qmc
#bounds enlarged for LHC

nvar=6
n_samplemany=300000 #for test

#LHC
sampler = qmc.LatinHypercube(d=nvar)     #d number of dimensions 
sample1 = sampler.random(n=n_samplemany) #n number of samples

scaler_sample1 = preprocessing.StandardScaler().fit(sample1)
scaled_sample1 = scaler_sample1.transform(sample1)

XLHC=np.zeros((n_samplemany,nvar))

for iv in np.arange(0,6):
    XLHC[:,iv]=np.min(bounds[iv])+(np.max(bounds[iv])-np.min(bounds[iv]))*sample1[:,iv]

#for iv in np.arange(0,6):
#    XLHC[:,iv]=np.min(var1[:,iv])+(np.max(var1[:,iv])-np.min(var1[:,iv]))*sample1[:,iv]
    
mean_XLHC, std_XLHC = gaussian_process_multi.predict(scaled_sample1, return_std=True)

print(gaussian_process_multi.get_params(deep=True))
#print('mean', np.size(mean_XLHC[0,:]))
#print('std', std_XLHC[:10] )

{'alpha': 1e-05, 'copy_X_train': True, 'kernel__k1': 1**2, 'kernel__k2': Matern(length_scale=1, nu=1.5), 'kernel__k1__constant_value': 1, 'kernel__k1__constant_value_bounds': (1e-05, 100000.0), 'kernel__k2__length_scale': 1.0, 'kernel__k2__length_scale_bounds': (1e-05, 100000.0), 'kernel__k2__nu': 1.5, 'kernel': 1**2 * Matern(length_scale=1, nu=1.5), 'n_restarts_optimizer': 20, 'normalize_y': False, 'optimizer': 'fmin_l_bfgs_b', 'random_state': None}


In [21]:
#intersection between both
def intersection(list_a, list_b):
    intt=[]
    for ii in list_a:
        intall=np.where(list_b==ii)[0]
        if np.size(intall)>0:
            intt.append(ii)
    return intt

In [22]:
def optimparameters(metric, T, T2):
    #FIND NROY PARAMETERS for 3 first outputs
    #T=0.5      # tolerance/treshold to implausibility metric
    tau=2   # tau + 1 = number of metrics for which the implausibility treshold should be reached, if tau=2 all 3 first !
    noutput1=3
    nplot=n_samplemany
    isnroy=np.zeros((nplot,noutput1))
    issumnroy=np.zeros((nplot,1))
    ismultinroy=np.zeros((nplot,1))

    #3 first outputs
    for imet in np.arange(0,3): #noutput): 
        intt = np.where(metric[:,imet]<T)
        isnroy[intt,imet] = 1

    issumnroy = isnroy.sum(axis=1)
    #print(issumnroy)
    #==np.min(abs(mean_X[:,6+ii]-ref_mean[ii])/ref_std[ii]))

    intTotalNroy=np.where(issumnroy>tau) # parameters should satisfies at least the implausibitly treshold for tau metrics 

    #print(np.size(intTotalNroy))
    #print(mean_X[:,6+ii][intt])

    #2 last outputs
    #T2=4.5     # tolerance/treshold to implausibility metric
    tau2=1   # tau + 1 = number of metrics for which the implausibility treshold should be reached, if tau2=1 all 2 last!
    noutput2=2
    nplot2=n_samplemany
    isnroy2=np.zeros((nplot,noutput2))
    issumnroy2=np.zeros((nplot,1))
    ismultinroy2=np.zeros((nplot,1))

    for imet in np.arange(3,5): #noutput): 
        intt = np.where(metric[:,imet]<T2)
        #print(imet,intt)
        isnroy2[intt,imet-3] = 1

    issumnroy2 = isnroy2.sum(axis=1)
    #==np.min(abs(mean_X[:,6+ii]-ref_mean[ii])/ref_std[ii]))
    intTotalNroy2=np.where(issumnroy2>tau2) # parameters should satisfies at least the implausibitly treshold for tau metrics 

    #print(np.size(intTotalNroy2))
    #print(Impl2)

    inter=intersection(intTotalNroy[0],intTotalNroy2[0])
    print(np.size(inter))
    
    return inter



In [24]:
noutput=5
nplot=n_samplemany
Impltot=np.zeros((nplot,noutput))
for imet in np.arange(0,noutput): #check all 5 metrics
    Impltot[:,imet] = abs(mean_XLHC[:,imet]-ref_mean[imet])/np.sqrt(ref_std[imet]**2+std_XLHC[:,imet]**2)

Abserror=np.zeros((nplot,noutput))
for imet in np.arange(0,noutput): #check all 5 metrics
    Abserror[:,imet] = abs(mean_XLHC[:,imet]-ref_mean[imet])#/np.sqrt(ref_std[ii]**2+std_XLHC[:,imet]**2)

In [25]:
#history matching: selected PPE
#the parameter values for parameter number ipara are given by: mean_XLHCMore[optim_impltotMore,ipara] and the outputs: mean_XLHC[optim_impltotMore,ioutput]

#threshold values
# for the 3 radiation metrics
T1=2.8
# for the 2 other metrics: cloud cover and water vapor path
T2=3.7 #0.2

#indices of the parameters that are kept 
optim_impltot=optimparameters(Impltot,T1,T2)
print('optim for Implausibility metric', np.size(optim_impltot),optim_impltot)


28
optim for Implausibility metric 28 [3932, 18480, 33490, 38012, 38748, 39445, 63038, 70802, 93950, 100691, 104536, 106490, 119843, 168657, 173708, 180017, 211959, 227016, 232437, 235485, 254148, 254453, 257570, 262117, 268261, 269869, 271441, 272659]
